# Proofread in eCREST

The files generated by this script will also be able to be opened in CREST original (though some information may be lost if using original CREST.py or .exe).

## Setup

Do the following two setup steps regardless of how you will be using this script. 

### 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new

from eCREST_cli import ecrest, import_settings

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


# USING THE CREST_JSON class

## Settings definitions

Whether you are converting from neuroglancer or creating a new reconstruction, the settings_dict parameters is needed to create CREST json files with correct formatting. 
- 'save_dir' : the directory where JSON files are saved 
- 'cred' and 'db_path' : specify the path to the agglomeration database file on your local computer. 

In [ ]:
settings_dict = {
    'save_dir' : '/Users/kperks/Documents/eCREST-local-files/in-progress',
    'db_path' : '/Users/kperks/Documents/eCREST-local-files/Mariela_bigquery_exports_agglo_v230111c_16_crest_proofreading_database.db',
    'max_num_base_added' : 1000,
    'cell_structures' : ['unknown','axon', 'basal dendrite', 'apical dendrite', 'dendrite', 'multiple'],
    'annotation_points' : ['exit volume', 'natural end', 'uncertain', 'pre-synaptic', 'post-synaptic']
}

### Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [2]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

## Proofread using eCREST



### 1. Create a crest_json object that launches a proofreading instance of neuroglancer


Initialize with either:
- (segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.
- (segment_id): a "main_base_id"
- (filepath): an existing CREST json file

#### NEW reconstruction from segment ID

If you wanted to start reconstructing a new cell from a main base segment, 
you would use the following code block to launch

In [ ]:
segment_id = 558129604
crest = ecrest(settings_dict,segment_id = segment_id, launch_viewer=True)

#### EDIT reconstruction from file

If you wanted to edit a reconstruction from an existing file, 
you would use the following code block to launch

In [3]:
# json_path = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
# filename = 'cell_graph_213605530__2023-03-29 22.49.21.json'

json_path = Path(settings_dict['save_dir'])
filename = 'cell_graph_386300356__2023-04-08 07.55.48.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

################
## You can also open using a full copy/pasted filepath 
## (instead of directory path and file name... which is better for looping over cells)

# filepath = r"C:\Users\PerksLab\Downloads\cell_graph_133378529__2023-03-27 14.03.02.json"
# crest = ecrest(settings_dict,filepath = filepath, launch_viewer=True)
########################

updating viewer status message: Current Base Segment Counts: unknown: 3402, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0


In [4]:
crest.version

0.3

In [6]:
crest.cell_structure_pos

1

at least entered the function
472174766 id loaded from click
['390757011', '130917920', '472176876', '472174766', '645024223', '556890831', '642719144', '562476330', '647254377', '473261074'] first 10 ds segments... just to check they exist
got downstream segs
if you are seeing this message, then a new cell part is being selected rather than unknonw
got new color
added this segment and ds to axon
changed location of downstream segs in cell_data base_segments dict
updated mtab
at least entered the function
377202918 id loaded from click
['368122335', '630252374', '624528555', '551227106', '624512143', '460803672', '613149212', '529565734', '550096846', '459659697'] first 10 ds segments... just to check they exist
got downstream segs
if you are seeing this message, then a new cell part is being selected rather than unknonw
got new color
added this segment and ds to apical dendrite
changed location of downstream segs in cell_data base_segments dict
updated mtab
at least entered the functi

In [ ]:
# crest.viewer.state.layers['base_segs'].segment_colors
base_seg = '377202918'
col = crest.viewer.state.layers['base_segs'].segment_colors
current_colour = col[int(base_seg)]
cell_part = crest.cell_structures[crest.cell_structure_pos]
new_colour = crest.chosen_seg_colours[cell_part]
new_colour

In [ ]:

downstream_segs = crest.get_ds_segs_of_certain_col(base_seg, current_colour)

In [ ]:
# downstream_segs

In [ ]:
crest.cell_data['base_segments'].keys()

In [ ]:
tmp = deepcopy(crest.cell_data['base_segments'])

In [ ]:
# for cs in tmp.keys():
#     if cs == cell_part:
#         tmp[cs].update(downstream_segs)
#     else:
#         tmp[cs] -= downstream_segs
for cs in crest.cell_data['base_segments'].keys():
    if cs == cell_part:
        crest.cell_data['base_segments'][cs].update(downstream_segs)
    else:
        crest.cell_data['base_segments'][cs] -= downstream_segs

In [ ]:
# tmp

In [ ]:
with crest.viewer.txn(overwrite=True) as s:
    for bs in downstream_segs:
        s.layers['base_segs'].segment_colors[int(bs)] = new_colour

In [ ]:
crest.update_seg_counts_msg()

### 2. SAVE YOUR WORK BEFORE CLOSING NEUROGLANCER! 

In [8]:
crest.save_cell_graph()

made it to saving points
appended points
made it to saving points
made it to saving points
made it to saving points
made it to saving points
Saved cell 386300356 reconstruction locally at 2023-04-08 14.27.32


If you want to re-write the file you opened instead of saving with a new timestamp in the filename, run the following code cell instead of the previous one.

In [ ]:
filepath = json_path / filename
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); 

### 3. CELL TYPING

If part of your job as a reconstructor is to identify cell types, then you can use the following blocks of code.  
First, check if it is already defined (and what the cell type was defined as).  


In [ ]:
# Assign which method you are using (manual or auto)
method = 'manual'

## Do not edit
crest.get_ctype(method)

If not defined (or defined incorrectly), then define it.
> OPTIONS: mg1, mg2, mgx, lg, lf, lx, mli, gc, gran, sg

In [ ]:
# Assign the cell type and which method you are using (manual or auto)
cell_type = ''
method = 'manual'

## Do not edit
crest.define_ctype(cell_type,method)

### DONT FORGET TO SAVE YOUR WORK! 

(step 2)

## Convert From Neuroglancer to eCREST

Run the following code cell to convert neuroglancer json files to eCREST json files. 

Uses "conversion_specs.json" to batch process conversion.

Conversion using "conversion_specs.json" expects:
- a folder of neuroglancer json files (with filenames standardized like in Google Drive)
- "dirname" is the folder containing neuroglancer json files to be converted
- that the "conversion_specs.json" is in the ```settings_dict['save_dir']``` key

### Batch

In [ ]:
conversion_specs_filename = "conversion_specs.json"

with open(Path(settings_dict['save_dir']) / conversion_specs_filename) as f:
    conversion_specs = json.load(f)

p = Path(conversion_specs['dirname'])

for cell_id, info in conversion_specs['cell_info'].items():
   
    f = info['filename']
    neuroglancer_layer_name = info['neuroglancer_layer_name']
    crest_layer_name = info['crest_layer_name']
  
    ## Get main_base_seg_ID from filename or from list of segment IDs
    main_base_id = f.split('_')[1] # gets the base segment ID from the name
    
    try:
        assert cell_id == main_base_id, f'cell id and filename do not match in conversion json; moving on to next cell without completing this one'
    except AssertionError as msg:
        print(msg)
        #add error message to json
        with open(settings_dict['save_dir'] / conversion_specs_filename, "r") as f:
            loaded = json.load(f)
        loaded['cell_info'][cell_id]['errors'].append(str(msg))
        with open(settings_dict['save_dir'] / conversion_specs_filename, "w") as f:
            json.dump(loaded, f, indent=4)
        continue
    
    ## Load the neuroglancer json
    print(f'you have selected cell {cell_id} to convert')
    
    with open(p / f, 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
        neuroglancer_data = json.load(myfile)

    print(f'Obtaining base_seg IDs from segmentation layer of neuroglancer json.')

    ## Obtain the list of base_segments from the neuroglancer json
    segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
    try:
        # add annotation layer
        
        base_segment_list_ng = segmentation_layer['segments']
    except TypeError as msg:
        print(msg, f': segmentation layer source is different; moving on to next cell without completing this one')
        #add error message to json
        with open(settings_dict['save_dir'] / conversion_specs_filename, "r") as f:
            loaded = json.load(f)
        loaded['cell_info'][cell_id]['errors'].append(str(msg) + f': segmentation layer source is different; moving on to next cell without completing this one')
        with open(settings_dict['save_dir'] / conversion_specs_filename, "w") as f:
            json.dump(loaded, f, indent=4)
        continue

    
    

    print(f'creating a crest_json object with no viewer for this cell')
    ## Create CREST instance with no viewer, segment_list, and segment_id
    crest = ecrest(settings_dict, segment_id = main_base_id, segment_list = base_segment_list_ng, launch_viewer=False)

    print(f'importing annotation layers from neuroglancer')
    ## Get annotations from neuroglancer -- iterate through one layer at a time to check for errors in layer names
    for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

        # get the 'layers' dictionary that has that name
        neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

        if neuroglancer_layer != None:
            if cl_ in crest.point_types:
                # add annotation layer
                crest.import_annotations(neuroglancer_data, [nl_], [cl_])
                print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
            else: 
                msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
                print(msg)
                #add error message to json
                with open(crest.save_dir / conversion_specs_filename, "r") as f:
                    loaded = json.load(f)
                loaded['cell_info'][cell_id]['errors'].append(str(msg))
                with open(crest.save_dir / conversion_specs_filename, "w") as f:
                    json.dump(loaded, f, indent=4)
        else:
            msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
            print(msg)
            #add error message to json
            with open(crest.save_dir / conversion_specs_filename, "r") as f:
                loaded = json.load(f)
            loaded['cell_info'][cell_id]['errors'].append(str(msg))
            with open(crest.save_dir / conversion_specs_filename, "w") as f:
                json.dump(loaded, f, indent=4)


    ## Save the cell_data as json
    print(f'saving cell {cell_id} with completed graph and annotations layers imported')
    crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

### Single file

Just make the "conversion_specs" file have one cell in it. The "batch" loop will still run on one cell. 

### Just annotations layer

If starting from scratch on a reconstruction is faster than converting the base_segs into a graph... but you want the annotations preserved.

In [ ]:
neuroglancer_layer_name = ['ends']
crest_layer_name = ['natural end']
neuroglancer_path = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/Complete_Reconstruction/NoSynapses/cell_558020412_type_LG_szm.json'

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)
# for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):
    # get the 'layers' dictionary that has that name
    neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

    if neuroglancer_layer != None:
        if cl_ in crest.point_types:
            # add annotation layer
            crest.import_annotations(neuroglancer_data, [nl_], [cl_])
            print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
        else: 
            msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
            print(msg)

    else:
        msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
        print(msg)


## Save the cell_data as json
print(f'saving cell {neuroglancer_path} with completed graph and annotations layers imported')
crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

In [ ]:
neuroglancer_layer

## Other...

### define cell type for a crest file

resaves as original file name (not with an updated timestamp)

In [ ]:
dirpath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
filepath = dirpath / 'cell_graph_307591597__2023-04-07 12.54.44.json'
cell_type = 'lf'

### 
crest = ecrest(settings_dict, filepath = filepath, launch_viewer=False);
crest.define_ctype(cell_type,'manual')
crest.get_ctype('manual') == cell_type
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); #rewrites the original, not with a new time stamp

check cell type in neuroglancer

In [ ]:
dirpath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
filepath = dirpath / 'cell_graph_213605530__2023-03-29 22.49.21.json'

crest = ecrest(settings_dict, filepath = filepath, launch_viewer=True)

In [ ]:
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); 

### get cell types of neuroglancer reconstructions into crest json files


In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

In [ ]:
crestpath = "/Volumes/GoogleDrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"
ngpath = "/Volumes/GoogleDrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/files_for_names"
ngfiles = [x.name for x in Path(ngpath).iterdir()]

In [ ]:
ctype_list = []
has_ctype = set()
all_cells = set()

for fname in sorted(list(Path(crestpath).iterdir())):
    if (fname.name[0]!='.') & (fname.is_file()):
        # display(fname.name)
        crest = ecrest(settings_dict, filepath = fname, launch_viewer=False);
        ngfile = list(filter(lambda x: cell.cell_data['metadata']['main_seg']['base'] in x, ngfiles))
        
        all_cells = all_cells | set({cell.cell_data['metadata']['main_seg']['base']})
        
        if len(ngfile)==1:
            ctype = ngfile[0].split('_')[3].lower()
            has_ctype = has_ctype | set({cell.cell_data['metadata']['main_seg']['base']})
        ctype_list.append(ctype)
        crest.define_ctype(ctype,'manual');
        crest.save_cell_graph(directory_path = fname.parent, file_name=fname.name, save_to_cloud=False);

In [ ]:
# make sure all crest cells have cell type definition from neuroglancer file name
all_cells-has_ctype

In [ ]:
# check cell type labels
list(unique(ctype_list))

### resave a json file with formatting for readability

In [ ]:
filepath = Path("D:\electric-fish\eCREST\CREST_settings.json")
with open(filepath, "r") as f:
    loaded = json.load(f)

with open(filepath, "w") as f:
    json.dump(loaded, f, indent=4)